In [23]:
from googleapiclient.discovery import build
from datetime import date
today = date.today()
import pandas as pd

api_key = "Enter You API Key"

youtube = build('youtube', 'v3', developerKey=api_key)


def get_channel_id(custom_url):
    # Get the channel ID using the custom URL (username)
    response = youtube.search().list(
        q=custom_url,
        part='snippet',
        type='channel',
        maxResults=1
    ).execute()

    if response['items']:
        return response['items'][0]['snippet']['channelId']
    else:
        return None




def get_channel_videos(api_key, channel_id):
    """Retrieves video details from a YouTube channel using the YouTube Data API.

    Args:
        api_key (str): Your YouTube Data API key.
        channel_id (str): The ID of the YouTube channel.

    Returns:
        list: A list of dictionaries containing video details, or None if an error occurs.
    """


    
    # Get the uploads playlist ID
    request = youtube.channels().list(
        part='contentDetails',
        id=channel_id
    )
    response = request.execute()

    uploads_playlist_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

    # Retrieve video details
    request = youtube.playlistItems().list(
        part='snippet',
        playlistId=uploads_playlist_id,
        maxResults=200  # Adjust as needed (maximum is 50)
    )
    while request is not None:
        response = request.execute()

        # Extract video details from each item
        for item in response['items']:
            video_id = item['snippet']['resourceId']['videoId']
            video_url = f"https://www.youtube.com/watch?v={video_id}"
            title = item['snippet']['title']
            published_at = item['snippet']['publishedAt']  # ISO 8601 formatted date

            # Access video duration using a separate request
            duration_request = youtube.videos().list(
                part='contentDetails',
                id=video_id
            )
            duration_response = duration_request.execute()
            duration = duration_response['items'][0]['contentDetails']['duration']

            # Parse duration from YouTube's format (PT#H#M#S)
            duration = duration.replace("PT",'')
            duration = duration.replace("H",' hour ')
            duration = duration.replace("M",' minutes ')
            duration = duration.replace("S",' seconds ')

            yield {
                'video_url': video_url,
                'title': title,
                'published_at': published_at,
                'duration': duration
            }

        request = youtube.playlistItems().list_next(request, response)


channel_id = get_channel_id('onlinepjofficial')
    
alldata = []
for video_info in get_channel_videos(api_key, channel_id):
    if video_info:
        video_url = video_info['video_url']
        title = video_info['title']
        published_at = video_info['published_at']
        durationn = video_info['duration']
        Query_time = f"{today.month}-{today.year}"
        rows = [video_url, title, published_at, durationn, Query_time]
        print(rows)
        alldata.append(rows)

df=pd.DataFrame(alldata,columns= ['video_url', 'title', 'published_at','duration','Query_time'])
df.to_csv("youtubedata2.csv")




